### Classification with CNN on MNIST

In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# data
mnist_train = torchvision.datasets.MNIST(root="./data", train=True, transform=torchvision.transforms.ToTensor(), download=True)
mnist_test = torchvision.datasets.MNIST(root="./data", train=False, transform=torchvision.transforms.ToTensor(), download=True)

In [3]:
# hyperparameters
num_epochs = 10
num_classes = 10
batch_size = 100
learning_rate = 1e-3

In [10]:
# data loader
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size, shuffle=True)

In [11]:
class CNN(torch.nn.Module):
    
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)            
        )
        self.fc = torch.nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [12]:
model = CNN(num_classes).to("cuda")
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
# Training
loss_arr = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to("cuda")
        labels = labels.to("cuda")
        
        # Forward
        output = model(images)
        loss = loss_fn(output, labels)
        loss_arr.append(loss.item())
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print("Epoch: ",epoch, loss.item())

Epoch:  0 0.02979021519422531
Epoch:  1 0.031935907900333405
Epoch:  2 0.026143372058868408
Epoch:  3 0.01500293891876936
Epoch:  4 0.0252029187977314
Epoch:  5 0.0038671207148581743
Epoch:  6 0.002515890635550022
Epoch:  7 0.003283255035057664
Epoch:  8 0.003896403359249234
Epoch:  9 0.011981670744717121


In [17]:
with torch.no_grad():
    c = 0
    t = 0
    for images, labels in test_loader:
        images = images.to("cuda")
        labels = labels.to("cuda")
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        t += labels.size(0)
        c += (predicted == labels).sum().item()
        
    print("Accuracy: ",100*c/t)

Accuracy:  99.02
